# Импорты библиотек

In [1]:
import sympy
import plotly.express as px
import numpy
from scipy.interpolate import lagrange

# Входные данные

In [2]:
x = sympy.symbols('x')

f = sympy.sin(x)
n = 10
a = 0
b = 2 * numpy.pi

# Определение $x_i$ и $f_i$

In [3]:
h = (b - a) / 10
x_points = []
y_points = []
for i in range(n + 1):
    x_points.append(a + i * h)
    y_points.append(float(f.subs(x, x_points[i])))

figure = px.line(x=x_points, y=y_points, markers=True)
figure.update_traces(name="f(x)", showlegend=True)
figure.show()

# Сплайны

## 1. Эрмитовы кубические сплайны

### Входные данные

In [4]:
y_1_points = []
for i in range(n + 1):
    y_1_points.append(float(sympy.diff(f, x).subs(x, x_points[i])))

figure = px.line(x=x_points, y=y_points, markers=True)
figure.update_traces(name="f(x)", showlegend=True)
figure.add_scatter(x=x_points, y=y_1_points, name="f'(x)")
figure.show()

### Определение

$\displaystyle 1) S(x) \in P_{3i}(x), \forall x \in [x_i; x_{i + 1}], i = \overline{0, n - 1} \\
2) S(x) \in C^1[a; b] \\
3) S(x_i) = f(x_i), S'(x_i) = f'(x_i), i = \overline{0, n} \\
\\
S(x) = y_i + y'_i (x - x_i) + a_i \frac{(x - x_i)^2}{2} + b_i \frac{(x - x_i)^3}{6}, \\
a_i = \frac{6}{h_{i + 1}} \left( \frac{y_{i + 1} - y_i}{h_{i + 1}} - \frac{2 y'_i + y'_{i + 1}}{3} \right), \\
b_i = \frac{12}{h_{i + 1}^2} \left( \frac{y'_i + y'_{i + 1}}{2} - \frac{y_{i + 1} - y_i}{h_{i + 1}} \right)
$

In [5]:
def ermith_cube_spline(i: int, x_points: list[float], y_points: list[float], y_1_points: list[float]) -> sympy.core.add.Add:
    h = abs(x_points[i + 1] - x_points[i])

    a_i = 6.0 / h * ((y_points[i + 1] - y_points[i]) / h - (y_1_points[i + 1] + 2 * y_1_points[i]) / 3.0)
    b_i = 12.0 / h ** 2 * ((y_1_points[i] + y_1_points[i + 1]) / 2.0 - (y_points[i + 1] - y_points[i]) / h)

    return y_points[i] + y_1_points[i] * (x - x_points[i]) + a_i * ((x - x_points[i]) ** 2) / 2.0 + b_i * ((x - x_points[i]) ** 3) / 6.0

### Визуализация

In [6]:
xs = numpy.linspace(a, b, 1000)
ys = sympy.lambdify(x, f, "numpy")(xs)

figure = px.line(x=xs, y=ys)
figure.update_traces(name="f(x)", showlegend=True)

splines = []
for i in range(n):
    S_i = ermith_cube_spline(i, x_points, y_points, y_1_points)
    splines.append(sympy.lambdify(x, S_i, "numpy"))

lagranges = []
for i in range(1, n - 2):
    L_i = lagrange(x_points[i - 1 : i + 3], y_points[i - 1 : i + 3])
    lagranges.append(L_i)

for i, L_i in enumerate(lagranges):
    xs_i = numpy.linspace(x_points[i], x_points[i + 1], int(1000 / n))
    ys_i = numpy.polyval(L_i, xs_i)

    figure.add_scatter(x=xs_i, y=ys_i, name=f"L_3(x)", showlegend=(i == 0), line=dict(color="red"))

xs_i = numpy.linspace(x_points[n - 3], x_points[n], int(1000 / (3 * n)))
ys_i = numpy.polyval(L_i, xs_i)
figure.add_scatter(x=xs_i, y=ys_i, name=f"L_3(x)", showlegend=False, line=dict(color="red"))

for i, S_i in enumerate(splines):
    xs_i = numpy.linspace(x_points[i], x_points[i + 1], int(1000 / n))
    ys_i = S_i(xs_i)

    figure.add_scatter(x=xs_i, y=ys_i, name=f"S(x)", showlegend=(i == 0), line=dict(color="green"))

for i in range(n + 1):
    figure.add_scatter(x=[x_points[i]], y=[y_points[i]], showlegend=(False), line=dict(color="blue"), mode="markers")

figure.show()

## 2. Нелокальные кубические сплайны

### Метод монотонной прогонки

In [7]:
def monotonous_reduction(A: numpy.array, B: numpy.array, C: numpy.array, F: numpy.array) -> numpy.array:
    n = len(A) - 1

    alpha = numpy.zeros(n + 1)
    beta = numpy.zeros(n + 1)
    U = numpy.zeros(n + 1)

    alpha[1] = -C[0] / B[0]
    beta[1] = F[0] / B[0]

    for i in range(1, n):
        alpha[i + 1] = -C[i] / (B[i] + alpha[i] * A[i])
        beta[i + 1] = (F[i] - beta[i] * A[i]) / (B[i] + alpha[i] * A[i])

    U[n] = (F[n] - beta[n] * A[n]) / (B[n] + alpha[n] * A[n])

    for i in range(n - 1, -1, -1):
        U[i] = alpha[i + 1] * U[i + 1] + beta[i + 1]

    return U

### Формула Вудбери

Алгоритм решения системы $Ax = b$, где $A = \left( \begin{array}{cccccccc} 
    B_0 & C_0 & 0 & 0 & \cdots & 0 & 0 & A_0 \\
    A_1 & B_1 & C_1 & 0 & \cdots & 0 & 0 & 0 \\ 
    0 & A_2 & B_2 & C_2 & \cdots & 0 & 0 & 0 \\ 
    \vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\ 
    0 & 0 & 0 & 0 & \cdots & A_{n - 1} & B_{n - 1} & C_{n - 1} \\ 
    C_n & 0 & 0 & 0 & \cdots & 0 & A_n & B_n
\end{array} \right)$:</br>
1) Построить $T = A - \Lambda$, где $\Lambda = \left( \begin{array}{ccc} 
    0 & \cdots & A_0 \\
    \vdots & \ddots & \vdots \\
    C_n & \cdots & 0
\end{array} \right)$
2) Решить $Ty = b$
3) Решить $Tz^{(1)} = e^{(1)}$, где $e^{(1)} = \left( \begin{array}{c}
    1 \\ 0 \\ \vdots \\ 0 \\ 0
\end{array} \right)$
4) Решить $Tz^{(2)} = e^{(n + 1)}$, где $e^{(n + 1)} = \left( \begin{array}{c}
    0 \\ 0 \\ \vdots \\ 0 \\ 1
\end{array} \right)$
5) Решить $Ma = v$, где $M = S + W$, $S = \left( \begin{array}{cc}
    0 & A_0 \\
    C_n & 0
\end{array} \right)$, $W = \left( \begin{array}{cc}
    z_{n + 1}^{1} & z_{n + 1}^{2} \\
    z_1^{1} & z_1^{2}
\end{array} \right)$, $v = \left( \begin{array}{c}
    y_{n + 1} \\ y_0
\end{array} \right)$
6) Получить $x = y - a_1 z^{(1)} - a_2 z^{(2)}$

In [8]:
def woodbury_reduction(A: numpy.array, B: numpy.array, C: numpy.array, F: numpy.array) -> numpy.array:
    N = len(A)

    e_1 = numpy.zeros(n + 1)
    e_N = numpy.zeros(n + 1)

    e_1[0] = 1
    e_N[N - 1] = 1

    T_A = A
    T_A[0] = 0

    T_C = C
    T_C[N - 1] = 0

    y = monotonous_reduction(T_A, B, T_C, F)
    z_1 = monotonous_reduction(A, B, C, e_1)
    z_2 = monotonous_reduction(A, B, C, e_N)

    S = numpy.array([
        [0, A[0]],
        [C[N - 1], 0]
    ])
    W = numpy.array([
        [z_1[N - 1], z_2[N - 1]],
        [z_1[0], z_2[0]]
    ])
    M = S + W
    v = numpy.array([y[N - 1], y[0]]).T
    a = numpy.linalg.solve(M, v)

    U = y - a[0] * z_1 - a[1] * z_2

    return U


### Определение

$\displaystyle 1) S(x) \in P_{3i}(x), \forall x \in [x_i; x_{i + 1}], i = \overline{0, n - 1} \\
2) S(x) \in C^2[a; b] \\
3) S(x_i) = f(x_i), i = \overline{0, n} \\
\\
S(x) = f(x_i) + c_i (x - x_i) + a_i \frac{(x - x_i)^2}{2} + b_i \frac{(x - x_i)^3}{6}, \forall x \in [x_i; x_{i + 1}]
$

In [9]:
def non_local_cube_spline(i: int, x_points: list[float], y_points: list[float], c_i: float, a_i: float, b_i: float) -> sympy.core.add.Add:
    return y_points[i] + c_i * (x - x_points[i]) + a_i * ((x - x_points[i]) ** 2) / 2.0 + b_i * ((x - x_points[i]) ** 3) / 6.0
    

### 2.1 Через наклоны

$\displaystyle S'(x_i) = m_i \\
\mu_i m_{i - 1} + 2 m_i + \lambda_i m_{i + 1} = 3 \left( \lambda_i \frac{y_{i + 1} - y_i}{h_{i + 1}} + \mu_i \frac{y_i - y_{i - 1}}{h_i} \right) = g_i, i = \overline{1, n - 1}, \\
\mu_i = \frac{h_{i + 1}}{h_i + h_{i + 1}},\, \lambda_i = \frac{h_i}{h_i + h_{i + 1}},\, \lambda_i + \mu_i = 1\\
\\
S(x) = f(x_i) + f'(x_i) (x - x_i) + a_i \frac{(x - x_i)^2}{2} + b_i \frac{(x - x_i)^3}{6}, \forall x \in [x_i; x_{i + 1}], \\
a_i = \frac{6}{h_{i + 1}} \left( -\frac{2 m_i + m_{i + 1}}{3} + \frac{y_{i + 1} - y_i}{h_{i + 1}} \right) \\
b_i = \frac{12}{h_{i + 1}^2} \left( \frac{m_i + m_{i + 1}}{2} - \frac{y_{i + 1} - y_i}{h_{i + 1}} \right)
$

In [10]:
def non_local_cube_spline_through_slopes(i: int, x_points: list[float], y_points: list[float], U: numpy.array) -> sympy.core.add.Add:
    h_i_plus_1 = x_points[i + 1] - x_points[i]

    a_i = 6.0 / h_i_plus_1 * (-(2 * U[i] + U[i + 1]) / 3.0 + (y_points[i + 1] - y_points[i]) / h_i_plus_1)
    b_i = 12.0 / h_i_plus_1 ** 2 * ((U[i] + U[i + 1]) / 2.0 - (y_points[i + 1] - y_points[i]) / h_i_plus_1)
    c_i = U[i]

    return non_local_cube_spline(i, x_points, y_points, c_i, a_i, b_i)

#### 2.1.1 1-е краевое условие

$\displaystyle S'(a) = f'(a),\, S'(b) = f'(b): \\
\left( \begin{array}{cccccccc}
    1 & 0 & 0 & 0 & \cdots & 0 & 0 & 0 \\
    \mu_1 & 2 & \lambda_1 & 0 & \cdots & 0 & 0 & 0 \\
    0 & \mu_2 & 2 & \lambda_2 & \cdots & 0 & 0 & 0 \\
    \vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
    0 & 0 & 0 & 0 & \cdots & \mu_{n - 1} & 2 & \lambda_{n - 1} \\
    0 & 0 & 0 & 0 & \cdots & 0 & 0 & 1
\end{array} \right) \left( \begin{array}{c}
    m_0 \\ m_1 \\ m_2 \\ \vdots \\ m_{n - 1} \\ m_n
\end{array} \right) = \left( \begin{array}{c}
    f'(a) \\ g_1 \\ g_2 \\ \vdots \\ g_{n - 1} \\ f'(b)
\end{array} \right)
$

In [11]:
def first_boundary_condition_for_slopes(x_points: list[float], y_points: list[float], f_0: float, f_n: float, n: int) -> numpy.array:
    A = numpy.zeros(n + 1)
    B = numpy.zeros(n + 1)
    C = numpy.zeros(n + 1)
    F = numpy.zeros(n + 1)

    lambdas = numpy.array([(x_points[i] - x_points[i - 1]) / (x_points[i + 1] - x_points[i - 1]) for i in range(1, n)])
    mus = numpy.array([(x_points[i + 1] - x_points[i]) / (x_points[i + 1] - x_points[i - 1]) for i in range(1, n)])
    g = numpy.array([3 * (lambdas[i - 1] * (y_points[i + 1] - y_points[i]) / (x_points[i + 1] - x_points[i]) + mus[i - 1] * (y_points[i] - y_points[i - 1]) / (x_points[i] - x_points[i - 1])) for i in range(1, n)])

    A[1:n] = mus
    C[1:n] = lambdas

    B[0] = 1
    B[1:n] = numpy.array([2] * (n - 1))
    B[n] = 1

    F[0] = f_0
    F[1:n] = g
    F[n] = f_n

    U = monotonous_reduction(A, B, C, F)
    return U

##### Визуализация

In [12]:
xs = numpy.linspace(a, b, 1000)
ys = sympy.lambdify(x, f, "numpy")(xs)

figure = px.line(x=xs, y=ys)
figure.update_traces(name="f(x)", showlegend=True)

splines = []
U = first_boundary_condition_for_slopes(x_points, y_points, y_1_points[0], y_1_points[-1], n)
for i in range(n):
    S_i = non_local_cube_spline_through_slopes(i, x_points, y_points, U)
    splines.append(sympy.lambdify(x, S_i, "numpy"))

lagranges = []
for i in range(1, n - 2):
    L_i = lagrange(x_points[i - 1 : i + 3], y_points[i - 1 : i + 3])
    lagranges.append(L_i)

for i, L_i in enumerate(lagranges):
    xs_i = numpy.linspace(x_points[i], x_points[i + 1], int(1000 / n))
    ys_i = numpy.polyval(L_i, xs_i)

    figure.add_scatter(x=xs_i, y=ys_i, name=f"L_3(x)", showlegend=(i == 0), line=dict(color="red"))

xs_i = numpy.linspace(x_points[n - 3], x_points[n], int(1000 / (3 * n)))
ys_i = numpy.polyval(L_i, xs_i)
figure.add_scatter(x=xs_i, y=ys_i, name=f"L_3(x)", showlegend=False, line=dict(color="red"))

for i, S_i in enumerate(splines):
    xs_i = numpy.linspace(x_points[i], x_points[i + 1], int(1000 / n))
    ys_i = S_i(xs_i)

    figure.add_scatter(x=xs_i, y=ys_i, name=f"S(x)", showlegend=(i == 0), line=dict(color="green"))

for i in range(n + 1):
    figure.add_scatter(x=[x_points[i]], y=[y_points[i]], showlegend=(False), line=dict(color="blue"), mode="markers")

figure.show()

#### 2.1.2 3-е краевое условие

$\displaystyle S''(a) = f''(a),\, S''(b) = f''(b): \\
\left( \begin{array}{cccccccc}
    2 & 1 & 0 & 0 & \cdots & 0 & 0 & 0 \\
    \mu_1 & 2 & \lambda_1 & 0 & \cdots & 0 & 0 & 0 \\
    0 & \mu_2 & 2 & \lambda_2 & \cdots & 0 & 0 & 0 \\
    \vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
    0 & 0 & 0 & 0 & \cdots & \mu_{n - 1} & 2 & \lambda_{n - 1} \\
    0 & 0 & 0 & 0 & \cdots & 0 & 1 & 2
\end{array} \right) \left( \begin{array}{c}
    m_0 \\ m_1 \\ m_2 \\ \vdots \\ m_{n - 1} \\ m_n
\end{array} \right) = \left( \begin{array}{c}
    g_0 \\ g_1 \\ g_2 \\ \vdots \\ g_{n - 1} \\ g_n
\end{array} \right) \\
g_0 = 3 \frac{y_1 - y_0}{h_1} - \frac{h_1}{2}f''(a) \\
g_n = 3 \frac{y_n - y_{n - 1}}{h_n} + \frac{h_n}{2} f''(b)
$

In [13]:
def third_boundary_condition_for_slopes(x_points: list[float], y_points: list[float], f_0: float, f_n: float, n: int) -> numpy.array:
    A = numpy.zeros(n + 1)
    B = numpy.array([2] * (n + 1))
    C = numpy.zeros(n + 1)
    F = numpy.zeros(n + 1)

    lambdas = numpy.array([(x_points[i] - x_points[i - 1]) / (x_points[i + 1] - x_points[i - 1]) for i in range(1, n)])
    mus = numpy.array([(x_points[i + 1] - x_points[i]) / (x_points[i + 1] - x_points[i - 1]) for i in range(1, n)])

    g = numpy.array([0] + [3 * (lambdas[i - 1] * (y_points[i + 1] - y_points[i]) / (x_points[i + 1] - x_points[i]) + mus[i - 1] * (y_points[i] - y_points[i - 1]) / (x_points[i] - x_points[i - 1])) for i in range(1, n)] + [0])
    g[0] = 3.0 * ((y_points[1] - y_points[0]) / (x_points[1] - x_points[0])) - (x_points[1] - x_points[0]) / 2.0 * f_0
    g[n] = 3.0 * ((y_points[n] - y_points[n - 1]) / (x_points[n] - x_points[n - 1])) + (x_points[n] - x_points[n - 1]) / 2.0 * f_n

    A[1:n] = mus
    A[n] = 1

    C[1:n] = lambdas
    C[0] = 1

    F = g

    U = monotonous_reduction(A, B, C, F)
    return U

##### Визуализация

In [14]:
xs = numpy.linspace(a, b, 1000)
ys = sympy.lambdify(x, f, "numpy")(xs)

figure = px.line(x=xs, y=ys)
figure.update_traces(name="f(x)", showlegend=True)

splines = []
U = third_boundary_condition_for_slopes(x_points, y_points, float(sympy.diff(f, x, 2).subs(x, x_points[0])), float(sympy.diff(f, x, 2).subs(x, x_points[-1])), n)
for i in range(n):
    S_i = non_local_cube_spline_through_slopes(i, x_points, y_points, U)
    splines.append(sympy.lambdify(x, S_i, "numpy"))

lagranges = []
for i in range(1, n - 2):
    L_i = lagrange(x_points[i - 1 : i + 3], y_points[i - 1 : i + 3])
    lagranges.append(L_i)

for i, L_i in enumerate(lagranges):
    xs_i = numpy.linspace(x_points[i], x_points[i + 1], int(1000 / n))
    ys_i = numpy.polyval(L_i, xs_i)

    figure.add_scatter(x=xs_i, y=ys_i, name=f"L_3(x)", showlegend=(i == 0), line=dict(color="red"))

xs_i = numpy.linspace(x_points[n - 3], x_points[n], int(1000 / (3 * n)))
ys_i = numpy.polyval(L_i, xs_i)
figure.add_scatter(x=xs_i, y=ys_i, name=f"L_3(x)", showlegend=False, line=dict(color="red"))

for i, S_i in enumerate(splines):
    xs_i = numpy.linspace(x_points[i], x_points[i + 1], int(1000 / n))
    ys_i = S_i(xs_i)

    figure.add_scatter(x=xs_i, y=ys_i, name=f"S(x)", showlegend=(i == 0), line=dict(color="green"))

for i in range(n + 1):
    figure.add_scatter(x=[x_points[i]], y=[y_points[i]], showlegend=(False), line=dict(color="blue"), mode="markers")

figure.show()

#### 2.1.3 5-е краевое условие (периодичность)

$\displaystyle f(a) = f(b),\, f(x_{n + 1}) = f(x_1),\, \ldots \\
m_0 = m_n,\, m_{n + 1} = m_1,\, \ldots \\
h_{n + 1} = h_1,\, h_{n + 2} = h_2,\, \ldots \\
\mu_{n + 1} = m_1,\, \mu_{n + 2} = \mu_2,\, \ldots: \\
\left( \begin{array}{cccccccc}
    2 & \lambda_1 & 0 & 0 & \cdots & 0 & 0 & \mu_1 \\
    \mu_2 & 2 & \lambda_2 & 0 & \cdots & 0 & 0 & 0 \\
    0 & \mu_3 & 2 & \lambda_3 & \cdots & 0 & 0 & 0 \\
    \vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
    0 & 0 & 0 & 0 & \cdots & \mu_{n - 1} & 2 & \lambda_{n - 1} \\
    \lambda_n & 0 & 0 & 0 & \cdots & 0 & \mu_n & 2
\end{array} \right) \left( \begin{array}{c}
    m_1 \\ m_2 \\ m_3 \\ \vdots \\ m_{n - 1} \\ m_n
\end{array} \right) = \left( \begin{array}{c}
    g_1 \\ g_2 \\ g_3 \\ \vdots \\ g_{n - 1} \\ g_n
\end{array} \right) \\
$

In [15]:
def fifth_boundary_condition_for_slopes(x_points: list[float], y_points: list[float], n: int) -> numpy.array:
    A = numpy.zeros(n)
    B = numpy.array([2] * n)
    C = numpy.zeros(n)
    F = numpy.zeros(n)
    U = numpy.zeros(n + 1)

    lambdas = numpy.array([(x_points[i] - x_points[i - 1]) / (x_points[i + 1] - x_points[i - 1]) for i in range(1, n)] + [0])
    lambdas[n - 1] = (x_points[n] - x_points[n - 1]) / (x_points[1] - x_points[0] + x_points[n] - x_points[n - 1])

    mus = numpy.array([0] + [(x_points[i + 1] - x_points[i]) / (x_points[i + 1] - x_points[i - 1]) for i in range(1, n)])
    mus[0] = (x_points[1] - x_points[0]) / (x_points[1] - x_points[0] + x_points[n] - x_points[n - 1])

    g = numpy.array([3 * (lambdas[i - 1] * (y_points[i + 1] - y_points[i]) / (x_points[i + 1] - x_points[i]) + mus[i - 1] * (y_points[i] - y_points[i - 1]) / (x_points[i] - x_points[i - 1])) for i in range(1, n)] + [0])
    g[n - 1] = 3.0 * (lambdas[n - 1] * (y_points[1] - y_points[n]) / (x_points[1] - x_points[0]) + mus[n - 1] * (y_points[n] - y_points[n - 1]) / (x_points[n] - x_points[n - 1]))

    A = mus
    C = lambdas
    F = g

    U[1: n + 1] = woodbury_reduction(A, B, C, F)
    U[0] = U[n]

    return U

##### Визуализация

In [16]:
xs = numpy.linspace(a, b, 1000)
ys = sympy.lambdify(x, f, "numpy")(xs)

figure = px.line(x=xs, y=ys)
figure.update_traces(name="f(x)", showlegend=True)

splines = []
U = fifth_boundary_condition_for_slopes(x_points, y_points, n)
for i in range(n):
    S_i = non_local_cube_spline_through_slopes(i, x_points, y_points, U)
    splines.append(sympy.lambdify(x, S_i, "numpy"))

lagranges = []
for i in range(1, n - 2):
    L_i = lagrange(x_points[i - 1 : i + 3], y_points[i - 1 : i + 3])
    lagranges.append(L_i)

for i, L_i in enumerate(lagranges):
    xs_i = numpy.linspace(x_points[i], x_points[i + 1], int(1000 / n))
    ys_i = numpy.polyval(L_i, xs_i)

    figure.add_scatter(x=xs_i, y=ys_i, name=f"L_3(x)", showlegend=(i == 0), line=dict(color="red"))

xs_i = numpy.linspace(x_points[n - 3], x_points[n], int(1000 / (3 * n)))
ys_i = numpy.polyval(L_i, xs_i)
figure.add_scatter(x=xs_i, y=ys_i, name=f"L_3(x)", showlegend=False, line=dict(color="red"))

for i, S_i in enumerate(splines):
    xs_i = numpy.linspace(x_points[i], x_points[i + 1], int(1000 / n))
    ys_i = S_i(xs_i)

    figure.add_scatter(x=xs_i, y=ys_i, name=f"S(x)", showlegend=(i == 0), line=dict(color="green"))

for i in range(n + 1):
    figure.add_scatter(x=[x_points[i]], y=[y_points[i]], showlegend=(False), line=dict(color="blue"), mode="markers")

figure.show()

#### 2.1.4 6-е краевое условие

$\displaystyle S'''(x_1 + 0) = S'''(x_1 - 0),\, S'''(x_{n - 1} + 0) = S'''(x_{n - 1} - 0): \\
\left( \begin{array}{cccccccc}
    1 + \gamma_1 & \gamma_1 & 0 & 0 & \cdots & 0 & 0 & 0 \\
    \mu_2 & 2 & \lambda_2 & 0 & \cdots & 0 & 0 & 0 \\
    0 & \mu_3 & 2 & \lambda_3 & \cdots & 0 & 0 & 0 \\
    \vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
    0 & 0 & 0 & 0 & \cdots & \mu_{n - 2} & 2 & \lambda_{n - 2} \\
    0 & 0 & 0 & 0 & \cdots & 0 & \gamma_n & 1 + \gamma_n
\end{array} \right) \left( \begin{array}{c}
    m_1 \\ m_2 \\ m_3 \\ \vdots \\ m_{n - 2} \\ m_{n - 1}
\end{array} \right) = \left( \begin{array}{c}
    \widehat{g}_1 \\ g_2 \\ g_3 \\ \vdots \\ g_{n - 2} \\ \widehat{g}_{n - 1}
\end{array} \right) \\
\widehat{g}_1 = \lambda_1 (3 + 2 \gamma_1) \frac{y_2 - y_1}{h_2} + \mu_1 \frac{y_1 - y_0}{h_1} \\
\widehat{g}_{n - 1} = \mu_{n - 1} (3 + 2 \gamma_n) \frac{y_{n - 1} - y_{n - 2}}{h_{n - 1}} + \lambda_{n - 1} \frac{y_n - y_{n - 1}}{h_n} \\
\gamma_1 = \frac{h_1}{h_2},\, \gamma_n = \frac{h_n}{h_{n - 1}} \\
\\
m_0 = \frac{\displaystyle 3 \left( \lambda_1 \frac{y_2 - y_1}{h_2} + \mu_1 \frac{y_1 - y_0}{h_1} \right) - 2m_1 - \lambda_1 m_2}{\mu_1} \\
\\
m_n = \frac{\displaystyle 3 \left( \lambda_{n - 1} \frac{y_n - y_{n - 1}}{h_n} + \mu_{n - 1} \frac{y_{n - 1} - y_{n - 2}}{h_{n - 1}} \right) - 2m_{n -1} - \mu_{n - 1} m_{n - 2}}{\lambda_{n - 1}}
$

In [17]:
def sixth_boundary_condition_for_slopes(x_points: list[float], y_points: list[float], n: int) -> numpy.array:
    A = numpy.zeros(n - 1)
    B = numpy.array([2] * (n - 1))
    C = numpy.zeros(n - 1)
    F = numpy.zeros(n - 1)
    U = numpy.zeros(n + 1)

    lambdas = numpy.array([(x_points[i] - x_points[i - 1]) / (x_points[i + 1] - x_points[i - 1]) for i in range(1, n)])
    mus = numpy.array([(x_points[i + 1] - x_points[i]) / (x_points[i + 1] - x_points[i - 1]) for i in range(1, n)])

    gamma_1 = (x_points[1] - x_points[0]) / (x_points[2] - x_points[1])
    gamma_n = (x_points[n] - x_points[n - 1]) / (x_points[n - 1] - x_points[n - 2])

    g = numpy.array([0] + [3 * (lambdas[i - 1] * (y_points[i + 1] - y_points[i]) / (x_points[i + 1] - x_points[i]) + mus[i - 1] * (y_points[i] - y_points[i - 1]) / (x_points[i] - x_points[i - 1])) for i in range(2, n - 1)] + [0])
    g[0] = lambdas[0] * (3.0 + 2.0 * gamma_1) * (y_points[2] - y_points[1]) / (x_points[2] - x_points[1]) + mus[0] * (y_points[1] - y_points[0]) / (x_points[1] - x_points[0])
    g[n - 2] = mus[n - 2] * (3.0 + 2.0 * gamma_n) * (y_points[n - 1] - y_points[n - 2]) / (x_points[n - 1] - x_points[n - 2]) + lambdas[n - 2] * (y_points[n] - y_points[n - 1]) / (x_points[n] - x_points[n - 1])

    A[1:-1] = mus[1:-1]
    A[-1] = gamma_n

    C[1:-1] = lambdas[1:-1]
    C[0] = gamma_1

    B[0] = 1 + gamma_1
    B[-1] = 1 + gamma_n

    F = g

    U[1:n] = monotonous_reduction(A, B, C, F)
    U[0] = (3.0 * (lambdas[0] * (y_points[2] - y_points[1]) / (x_points[2] - x_points[1]) + mus[0] * (y_points[1] - y_points[0]) / (x_points[1] - x_points[0])) - 2.0 * U[1] - lambdas[0] * U[2]) / mus[0]
    U[n] = (3.0 * (lambdas[n - 2] * (y_points[n] - y_points[n - 1]) / (x_points[n] - x_points[n - 1]) + mus[n - 2] * (y_points[n - 1] - y_points[n - 2]) / (x_points[n - 1] - x_points[n - 2])) - 2.0 * U[n - 1] - mus[n - 2] * U[n - 2]) / lambdas[n - 2]

    return U

##### Визуализация

In [18]:
xs = numpy.linspace(a, b, 1000)
ys = sympy.lambdify(x, f, "numpy")(xs)

figure = px.line(x=xs, y=ys)
figure.update_traces(name="f(x)", showlegend=True)

splines = []
U = sixth_boundary_condition_for_slopes(x_points, y_points, n)
for i in range(n):
    S_i = non_local_cube_spline_through_slopes(i, x_points, y_points, U)
    splines.append(sympy.lambdify(x, S_i, "numpy"))

lagranges = []
for i in range(1, n - 2):
    L_i = lagrange(x_points[i - 1 : i + 3], y_points[i - 1 : i + 3])
    lagranges.append(L_i)

for i, L_i in enumerate(lagranges):
    xs_i = numpy.linspace(x_points[i], x_points[i + 1], int(1000 / n))
    ys_i = numpy.polyval(L_i, xs_i)

    figure.add_scatter(x=xs_i, y=ys_i, name=f"L_3(x)", showlegend=(i == 0), line=dict(color="red"))

xs_i = numpy.linspace(x_points[n - 3], x_points[n], int(1000 / (3 * n)))
ys_i = numpy.polyval(L_i, xs_i)
figure.add_scatter(x=xs_i, y=ys_i, name=f"L_3(x)", showlegend=False, line=dict(color="red"))

for i, S_i in enumerate(splines):
    xs_i = numpy.linspace(x_points[i], x_points[i + 1], int(1000 / n))
    ys_i = S_i(xs_i)

    figure.add_scatter(x=xs_i, y=ys_i, name=f"S(x)", showlegend=(i == 0), line=dict(color="green"))

for i in range(n + 1):
    figure.add_scatter(x=[x_points[i]], y=[y_points[i]], showlegend=(False), line=dict(color="blue"), mode="markers")

figure.show()

### 2.2 Через моменты

$\displaystyle S''(x_i) = M_i \\
\lambda_i M_{i - 1} + 2 m_i + \mu_i M_{i + 1} = 6 \left( \mu_i \frac{y_{i + 1} - y_i}{h_{i + 1}^2} - \lambda_i \frac{y_i - y_{i - 1}}{h_i^2} \right) = g_i, i = \overline{1, n - 1}, \\
\mu_i = \frac{h_{i + 1}}{h_i + h_{i + 1}},\, \lambda_i = \frac{h_i}{h_i + h_{i + 1}},\, \lambda_i + \mu_i = 1\\
\\
S(x) = f(x_i) + c_i(x_i) (x - x_i) + a_i \frac{(x - x_i)^2}{2} + b_i \frac{(x - x_i)^3}{6}, \forall x \in [x_i; x_{i + 1}], \\
a_i = M_i \\
b_i = \frac{M_{i + 1} - M_i}{h_{i + 1}} \\
c_i = \frac{y_{i + 1} - y_i}{h_{i + 1}} - \frac{h_{i + 1}}{6} (M_{i + 1} + 2 M_i)
$

In [19]:
def non_local_cube_spline_through_moments(i: int, x_points: list[float], y_points: list[float], U: numpy.array) -> sympy.core.add.Add:
    h_i_plus_1 = x_points[i + 1] - x_points[i]

    a_i = U[i]
    b_i = (U[i + 1] - U[i]) / h_i_plus_1
    c_i = (y_points[i + 1] - y_points[i]) / h_i_plus_1 - h_i_plus_1 / 6.0 * (U[i + 1] + 2.0 * U[i])

    return non_local_cube_spline(i, x_points, y_points, c_i, a_i, b_i)

#### 2.2.1 1-е краевое условие

$\displaystyle S'(a) = f'(a),\, S'(b) = f'(b): \\
\left( \begin{array}{cccccccc}
    2 & 1 & 0 & 0 & \cdots & 0 & 0 & 0 \\
    \lambda_1 & 2 & \mu_1 & 0 & \cdots & 0 & 0 & 0 \\
    0 & \lambda_2 & 2 & \mu_2 & \cdots & 0 & 0 & 0 \\
    \vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
    0 & 0 & 0 & 0 & \cdots & \lambda_{n - 1} & 2 & \mu_{n - 1} \\
    0 & 0 & 0 & 0 & \cdots & 0 & 1 & 2
\end{array} \right) \left( \begin{array}{c}
    M_0 \\ M_1 \\ M_2 \\ \vdots \\ M_{n - 1} \\ M_n
\end{array} \right) = \left( \begin{array}{c}
    g_0 \\ g_1 \\ g_2 \\ \vdots \\ g_{n - 1} \\ g_n
\end{array} \right) \\
g_0 = 6 \frac{y_1 - y_0}{h_1^2} - \frac{6}{h_1}f'(a) \\
g_n = \frac{6}{h_n}f'(b) - 6 \frac{y_n - y_{n - 1}}{h_n^2}
$

In [20]:
def first_boundary_condition_for_moments(x_points: list[float], y_points: list[float], f_0: float, f_n: float, n: int) -> numpy.array:
    A = numpy.zeros(n + 1)
    B = numpy.array([2] * (n + 1))
    C = numpy.zeros(n + 1)
    F = numpy.zeros(n + 1)

    lambdas = numpy.array([(x_points[i] - x_points[i - 1]) / (x_points[i + 1] - x_points[i - 1]) for i in range(1, n)])
    mus = numpy.array([(x_points[i + 1] - x_points[i]) / (x_points[i + 1] - x_points[i - 1]) for i in range(1, n)])

    g = numpy.array([0] + [6.0 * (mus[i - 1] * (y_points[i + 1] - y_points[i]) / (x_points[i + 1] - x_points[i]) ** 2 - lambdas[i - 1] * (y_points[i] - y_points[i - 1]) / (x_points[i] - x_points[i - 1]) ** 2) for i in range(1, n)] + [0])
    g[0] = 6.0 * ((y_points[1] - y_points[0]) / (x_points[1] - x_points[0]) ** 2) - 6.0 / (x_points[1] - x_points[0]) * f_0
    g[n] = 6.0 / (x_points[n] - x_points[n - 1]) * f_n - 6.0 * ((y_points[n] - y_points[n - 1]) / (x_points[n] - x_points[n - 1]) ** 2)

    A[1:n] = lambdas
    A[n] = 1

    C[1:n] = mus
    C[0] = 1

    F = g

    U = monotonous_reduction(A, B, C, F)
    return U

##### Визуализация

In [21]:
xs = numpy.linspace(a, b, 1000)
ys = sympy.lambdify(x, f, "numpy")(xs)

figure = px.line(x=xs, y=ys)
figure.update_traces(name="f(x)", showlegend=True)

splines = []
U = first_boundary_condition_for_moments(x_points, y_points, y_1_points[0], y_1_points[-1], n)
for i in range(n):
    S_i = non_local_cube_spline_through_moments(i, x_points, y_points, U)
    splines.append(sympy.lambdify(x, S_i, "numpy"))

lagranges = []
for i in range(1, n - 2):
    L_i = lagrange(x_points[i - 1 : i + 3], y_points[i - 1 : i + 3])
    lagranges.append(L_i)

for i, L_i in enumerate(lagranges):
    xs_i = numpy.linspace(x_points[i], x_points[i + 1], int(1000 / n))
    ys_i = numpy.polyval(L_i, xs_i)

    figure.add_scatter(x=xs_i, y=ys_i, name=f"L_3(x)", showlegend=(i == 0), line=dict(color="red"))

xs_i = numpy.linspace(x_points[n - 3], x_points[n], int(1000 / (3 * n)))
ys_i = numpy.polyval(L_i, xs_i)
figure.add_scatter(x=xs_i, y=ys_i, name=f"L_3(x)", showlegend=False, line=dict(color="red"))

for i, S_i in enumerate(splines):
    xs_i = numpy.linspace(x_points[i], x_points[i + 1], int(1000 / n))
    ys_i = S_i(xs_i)

    figure.add_scatter(x=xs_i, y=ys_i, name=f"S(x)", showlegend=(i == 0), line=dict(color="green"))

for i in range(n + 1):
    figure.add_scatter(x=[x_points[i]], y=[y_points[i]], showlegend=(False), line=dict(color="blue"), mode="markers")

figure.show()

#### 2.2.2 3-е краевое условие

$\displaystyle S''(a) = f''(a),\, S''(b) = f''(b): \\
\left( \begin{array}{cccccccc}
    1 & 0 & 0 & 0 & \cdots & 0 & 0 & 0 \\
    \lambda_1 & 2 & \mu_1 & 0 & \cdots & 0 & 0 & 0 \\
    0 & \lambda_2 & 2 & \mu_2 & \cdots & 0 & 0 & 0 \\
    \vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
    0 & 0 & 0 & 0 & \cdots & \lambda_{n - 1} & 2 & \mu_{n - 1} \\
    0 & 0 & 0 & 0 & \cdots & 0 & 0 & 1
\end{array} \right) \left( \begin{array}{c}
    M_0 \\ M_1 \\ M_2 \\ \vdots \\ M_{n - 1} \\ M_n
\end{array} \right) = \left( \begin{array}{c}
    f''(a) \\ g_1 \\ g_2 \\ \vdots \\ g_{n - 1} \\ f''(b)
\end{array} \right)
$

In [22]:
def third_boundary_condition_for_moments(x_points: list[float], y_points: list[float], f_0: float, f_n: float, n: int) -> numpy.array:
    A = numpy.zeros(n + 1)
    B = numpy.zeros(n + 1)
    C = numpy.zeros(n + 1)
    F = numpy.zeros(n + 1)

    lambdas = numpy.array([(x_points[i] - x_points[i - 1]) / (x_points[i + 1] - x_points[i - 1]) for i in range(1, n)])
    mus = numpy.array([(x_points[i + 1] - x_points[i]) / (x_points[i + 1] - x_points[i - 1]) for i in range(1, n)])
    g = numpy.array([6.0 * (mus[i - 1] * (y_points[i + 1] - y_points[i]) / (x_points[i + 1] - x_points[i]) ** 2 - lambdas[i - 1] * (y_points[i] - y_points[i - 1]) / (x_points[i] - x_points[i - 1]) ** 2) for i in range(1, n)])

    A[1:n] = lambdas
    C[1:n] = mus

    B[0] = 1
    B[1:n] = numpy.array([2] * (n - 1))
    B[n] = 1

    F[0] = f_0
    F[1:n] = g
    F[n] = f_n

    U = monotonous_reduction(A, B, C, F)
    return U

##### Визуализация

In [23]:
xs = numpy.linspace(a, b, 1000)
ys = sympy.lambdify(x, f, "numpy")(xs)

figure = px.line(x=xs, y=ys)
figure.update_traces(name="f(x)", showlegend=True)

splines = []
U = third_boundary_condition_for_moments(x_points, y_points, float(sympy.diff(f, x, 2).subs(x, x_points[0])), float(sympy.diff(f, x, 2).subs(x, x_points[-1])), n)
for i in range(n):
    S_i = non_local_cube_spline_through_moments(i, x_points, y_points, U)
    splines.append(sympy.lambdify(x, S_i, "numpy"))

lagranges = []
for i in range(1, n - 2):
    L_i = lagrange(x_points[i - 1 : i + 3], y_points[i - 1 : i + 3])
    lagranges.append(L_i)

for i, L_i in enumerate(lagranges):
    xs_i = numpy.linspace(x_points[i], x_points[i + 1], int(1000 / n))
    ys_i = numpy.polyval(L_i, xs_i)

    figure.add_scatter(x=xs_i, y=ys_i, name=f"L_3(x)", showlegend=(i == 0), line=dict(color="red"))

xs_i = numpy.linspace(x_points[n - 3], x_points[n], int(1000 / (3 * n)))
ys_i = numpy.polyval(L_i, xs_i)
figure.add_scatter(x=xs_i, y=ys_i, name=f"L_3(x)", showlegend=False, line=dict(color="red"))

for i, S_i in enumerate(splines):
    xs_i = numpy.linspace(x_points[i], x_points[i + 1], int(1000 / n))
    ys_i = S_i(xs_i)

    figure.add_scatter(x=xs_i, y=ys_i, name=f"S(x)", showlegend=(i == 0), line=dict(color="green"))

for i in range(n + 1):
    figure.add_scatter(x=[x_points[i]], y=[y_points[i]], showlegend=(False), line=dict(color="blue"), mode="markers")

figure.show()

#### 2.2.3 5-е краевое условие (периодичность)

$\displaystyle f(a) = f(b),\, f(x_{n + 1}) = f(x_1),\, \ldots \\
m_0 = m_n,\, m_{n + 1} = m_1,\, \ldots \\
h_{n + 1} = h_1,\, h_{n + 2} = h_2,\, \ldots \\
\mu_{n + 1} = m_1,\, \mu_{n + 2} = \mu_2,\, \ldots: \\
\left( \begin{array}{cccccccc}
    2 & \mu_1 & 0 & 0 & \cdots & 0 & 0 & \lambda_1 \\
    \lambda_2 & 2 & \mu_2 & 0 & \cdots & 0 & 0 & 0 \\
    0 & \lambda_3 & 2 & \mu_3 & \cdots & 0 & 0 & 0 \\
    \vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
    0 & 0 & 0 & 0 & \cdots & \lambda_{n - 1} & 2 & \mu_{n - 1} \\
    \mu_n & 0 & 0 & 0 & \cdots & 0 & \lambda_n & 2
\end{array} \right) \left( \begin{array}{c}
    M_1 \\ M_2 \\ M_3 \\ \vdots \\ M_{n - 1} \\ M_n
\end{array} \right) = \left( \begin{array}{c}
    g_1 \\ g_2 \\ g_3 \\ \vdots \\ g_{n - 1} \\ g_n
\end{array} \right) \\
$

In [24]:
def fifth_boundary_condition_for_moments(x_points: list[float], y_points: list[float], n: int) -> numpy.array:
    A = numpy.zeros(n)
    B = numpy.array([2] * n)
    C = numpy.zeros(n)
    F = numpy.zeros(n)
    U = numpy.zeros(n + 1)

    lambdas = numpy.array([(x_points[i] - x_points[i - 1]) / (x_points[i + 1] - x_points[i - 1]) for i in range(1, n)] + [0])
    lambdas[n - 1] = (x_points[n] - x_points[n - 1]) / (x_points[1] - x_points[0] + x_points[n] - x_points[n - 1])

    mus = numpy.array([0] + [(x_points[i + 1] - x_points[i]) / (x_points[i + 1] - x_points[i - 1]) for i in range(1, n)])
    mus[0] = (x_points[1] - x_points[0]) / (x_points[1] - x_points[0] + x_points[n] - x_points[n - 1])

    g = numpy.array([6.0 * (mus[i - 1] * (y_points[i + 1] - y_points[i]) / (x_points[i + 1] - x_points[i]) ** 2 - lambdas[i - 1] * (y_points[i] - y_points[i - 1]) / (x_points[i] - x_points[i - 1]) ** 2) for i in range(1, n)] + [0])
    g[n - 1] = 6.0 * (mus[n - 1] * (y_points[1] - y_points[n]) / (x_points[1] - x_points[0]) ** 2 - lambdas[n - 1] * (y_points[n] - y_points[n - 1]) / (x_points[n] - x_points[n - 1]) ** 2)

    A = lambdas
    C = mus
    F = g

    U[1: n + 1] = woodbury_reduction(A, B, C, F)
    U[0] = U[n]

    return U

##### Визуализация

In [25]:
xs = numpy.linspace(a, b, 1000)
ys = sympy.lambdify(x, f, "numpy")(xs)

figure = px.line(x=xs, y=ys)
figure.update_traces(name="f(x)", showlegend=True)

splines = []
U = fifth_boundary_condition_for_moments(x_points, y_points, n)
for i in range(n):
    S_i = non_local_cube_spline_through_moments(i, x_points, y_points, U)
    splines.append(sympy.lambdify(x, S_i, "numpy"))

lagranges = []
for i in range(1, n - 2):
    L_i = lagrange(x_points[i - 1 : i + 3], y_points[i - 1 : i + 3])
    lagranges.append(L_i)

for i, L_i in enumerate(lagranges):
    xs_i = numpy.linspace(x_points[i], x_points[i + 1], int(1000 / n))
    ys_i = numpy.polyval(L_i, xs_i)

    figure.add_scatter(x=xs_i, y=ys_i, name=f"L_3(x)", showlegend=(i == 0), line=dict(color="red"))

xs_i = numpy.linspace(x_points[n - 3], x_points[n], int(1000 / (3 * n)))
ys_i = numpy.polyval(L_i, xs_i)
figure.add_scatter(x=xs_i, y=ys_i, name=f"L_3(x)", showlegend=False, line=dict(color="red"))

for i, S_i in enumerate(splines):
    xs_i = numpy.linspace(x_points[i], x_points[i + 1], int(1000 / n))
    ys_i = S_i(xs_i)

    figure.add_scatter(x=xs_i, y=ys_i, name=f"S(x)", showlegend=(i == 0), line=dict(color="green"))

for i in range(n + 1):
    figure.add_scatter(x=[x_points[i]], y=[y_points[i]], showlegend=(False), line=dict(color="blue"), mode="markers")

figure.show()

#### 2.2.4 6-е краевое условие

$\displaystyle S'''(x_1 + 0) = S'''(x_1 - 0),\, S'''(x_{n - 1} + 0) = S'''(x_{n - 1} - 0): \\
\left( \begin{array}{cccccccc}
    -\gamma_1 - 2 & \mu_1 \gamma_1^2 - \mu_1 & 0 & 0 & \cdots & 0 & 0 & 0 \\
    \lambda_2 & 2 & \mu_2 & 0 & \cdots & 0 & 0 & 0 \\
    0 & \lambda_3 & 2 & \mu_3 & \cdots & 0 & 0 & 0 \\
    \vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
    0 & 0 & 0 & 0 & \cdots & \lambda_{n - 2} & 2 & \mu_{n - 2} \\
    0 & 0 & 0 & 0 & \cdots & 0 & \lambda_{n - 1} \gamma_n^2 - \lambda_{n - 1} & -\gamma_n - 2
\end{array} \right) \left( \begin{array}{c}
    M_1 \\ M_2 \\ M_3 \\ \vdots \\ M_{n - 2} \\ M_{n - 1}
\end{array} \right) = \left( \begin{array}{c}
    \widehat{g}_1 \\ g_2 \\ g_3 \\ \vdots \\ g_{n - 2} \\ \widehat{g}_{n - 1}
\end{array} \right) \\
\widehat{g}_1 = -6 \left( \mu_1 \frac{y_2 - y_1}{h_2^2} - \lambda_1 \frac{y_1 - y_0}{h_1^2} \right) \\
\widehat{g}_{n - 1} = -6 \left( \mu_{n - 1} \frac{y_n - y_{n - 1}}{h_n^2} - \lambda_{n - 1} \frac{y_{n - 1} - y_{n - 2}}{h_{n - 1}^2} \right) \\
\gamma_1 = \frac{h_1}{h_2},\, \gamma_n = \frac{h_n}{h_{n - 1}} \\
\\
M_0 = M_1 (1 + \gamma_1) - \gamma_1 M_2 \\
\\
M_n = M_{n - 1} (1 + \gamma_n) - \gamma_n M_{n - 2}
$

In [26]:
def sixth_boundary_condition_for_moments(x_points: list[float], y_points: list[float], n: int) -> numpy.array:
    A = numpy.zeros(n - 1)
    B = numpy.array([2] * (n - 1))
    C = numpy.zeros(n - 1)
    F = numpy.zeros(n - 1)
    U = numpy.zeros(n + 1)

    lambdas = numpy.array([(x_points[i] - x_points[i - 1]) / (x_points[i + 1] - x_points[i - 1]) for i in range(1, n)])
    mus = numpy.array([(x_points[i + 1] - x_points[i]) / (x_points[i + 1] - x_points[i - 1]) for i in range(1, n)])

    gamma_1 = (x_points[1] - x_points[0]) / (x_points[2] - x_points[1])
    gamma_n = (x_points[n] - x_points[n - 1]) / (x_points[n - 1] - x_points[n - 2])

    g = numpy.array([0] + [6.0 * (mus[i - 1] * (y_points[i + 1] - y_points[i]) / (x_points[i + 1] - x_points[i]) ** 2 - lambdas[i - 1] * (y_points[i] - y_points[i - 1]) / (x_points[i] - x_points[i - 1]) ** 2) for i in range(2, n - 1)] + [0])
    g[0] = -6.0 * (mus[0] * (y_points[2] - y_points[1]) / (x_points[2] - x_points[1]) ** 2 + lambdas[0] * (y_points[1] - y_points[0]) / (x_points[1] - x_points[0]) ** 2)
    g[n - 2] = -6.0 * (mus[n - 2] * (y_points[n] - y_points[n - 1]) / (x_points[n] - x_points[n - 1]) ** 2 + lambdas[n - 2] * (y_points[n - 1] - y_points[n - 2]) / (x_points[n - 1] - x_points[n - 2]) ** 2)
    A[1:-1] = lambdas[1:-1]
    A[-1] = lambdas[n - 2] * gamma_n ** 2 - lambdas[n - 2]

    C[1:-1] = mus[1:-1]
    C[0] = mus[0] * gamma_1 ** 2 - mus[0]

    B[0] = -gamma_1 - 2
    B[-1] = -gamma_n - 2

    F = g

    U[1:n] = monotonous_reduction(A, B, C, F)
    U[0] = U[1] * (1 + gamma_1) - gamma_1 * U[2]
    U[n] = U[n - 1] * (1 + gamma_n) - gamma_n * U[n - 2]

    return U

In [27]:
xs = numpy.linspace(a, b, 1000)
ys = sympy.lambdify(x, f, "numpy")(xs)

figure = px.line(x=xs, y=ys)
figure.update_traces(name="f(x)", showlegend=True)

splines = []
U = sixth_boundary_condition_for_moments(x_points, y_points, n)
for i in range(n):
    S_i = non_local_cube_spline_through_moments(i, x_points, y_points, U)
    splines.append(sympy.lambdify(x, S_i, "numpy"))

lagranges = []
for i in range(1, n - 2):
    L_i = lagrange(x_points[i - 1 : i + 3], y_points[i - 1 : i + 3])
    lagranges.append(L_i)

for i, L_i in enumerate(lagranges):
    xs_i = numpy.linspace(x_points[i], x_points[i + 1], int(1000 / n))
    ys_i = numpy.polyval(L_i, xs_i)

    figure.add_scatter(x=xs_i, y=ys_i, name=f"L_3(x)", showlegend=(i == 0), line=dict(color="red"))

xs_i = numpy.linspace(x_points[n - 3], x_points[n], int(1000 / (3 * n)))
ys_i = numpy.polyval(L_i, xs_i)
figure.add_scatter(x=xs_i, y=ys_i, name=f"L_3(x)", showlegend=False, line=dict(color="red"))

for i, S_i in enumerate(splines):
    xs_i = numpy.linspace(x_points[i], x_points[i + 1], int(1000 / n))
    ys_i = S_i(xs_i)

    figure.add_scatter(x=xs_i, y=ys_i, name=f"S(x)", showlegend=(i == 0), line=dict(color="green"))

for i in range(n + 1):
    figure.add_scatter(x=[x_points[i]], y=[y_points[i]], showlegend=(False), line=dict(color="blue"), mode="markers")

figure.show()